# Loading DICOM images to python

In [34]:
import os
import numpy as np
import pandas as pd
import mudicom
import matplotlib.pyplot as plt
% matplotlib inline

## Labels

In [2]:
visual = pd.read_csv("../Sample/visual.csv", index_col=1)

In [3]:
labels = [x for x in visual.columns 
          if (x.startswith('E') or x.startswith('Dato')) 
          and not x.startswith('Ens')]
y = visual[labels]
y.head()

,Emf.L1,ERU.L1,ERM.L1,ERL.L1,ELU.L1,ELM.L1,ELL.L1,EPre.L1,Emf.M1,ERU.M1,...,ELL.L2,EPre.L2,Emf.M2,ERU.M2,ERM.M2,ERL.M2,ELU.M2,ELM.M2,ELL.M2,EPre.M2
Dato.1,,,,,,,,,,,,,,,,,,,,,
2005-03-15,1,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


In [4]:
y[["ERU.L1", "ERU.L2", "ERU.M1", "ERU.M2"]]

,ERU.L1,ERU.L2,ERU.M1,ERU.M2
Dato.1,,,,
2005-03-15,0,0,0,0


**Legend**:
* Emf $\in$ {1 = no emphysema, 2 = emphysema}
* ERU, ERM, ERL, ELU, ELM, ELL $\in$ {0 = 0%, 1 = 1-5%, 2 = 6-25%, 3 = 26-50%, 4 = 51-75%, 5 = 76-100%} (udbreddelse)
* EPre $\in$ {0 = no emphysema, 1 = panlobular, 2 = centrilobular, 3 = paraseptal, 4 = mixed} (emfysem-type)

## Input data

In [35]:
dicum = mudicom.load("../Sample/vol1235_20050315-D-1-carina_thirds.dcm")

The mask:

In [6]:
mask = dicum.image.numpy
mask.shape

(399, 512, 512)

6 = ERU (right upper):

In [7]:
np.unique(mask)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.])

The actual CT-scan:

In [8]:
dicum = mudicom.load("../Sample/vol1235_20050315-D-1.dcm")
scan = dicum.image.numpy
scan.shape

(399, 512, 512)

In [9]:
np.unique(scan)

array([-1000.,  -999.,  -998., ...,  1929.,  1939.,  2022.])

Some dicum metadata:

In [10]:
{x.name: x.value for x in dicum.read()}

{'Acquisition Date': '20050315',
 'Acquisition Time': '160023.000',
 'Bits Allocated': '16',
 'Bits Stored': '16',
 'Columns': '512',
 'Content Date': '20050315',
 'Content Time': '160034.780',
 'Conversion Type': 'SYN',
 'Convolution Kernel': 'D',
 'Data Collection Diameter': '400',
 'Derivation Description': 'From ELSCINT DICOM Lossless Rice by DICOMatic 1.8 rev-5b',
 'Exposure': '40',
 'File Meta Information Group Length': '254',
 'Filter Type': 'D',
 'Frame of Reference UID': '1.2.826.0.1.3680043.2.1125.1.113349180606.2011100717574376478',
 'Gantry/Detector Tilt': '0',
 'Generic Group Length': '86418236',
 'High Bit': '15',
 'Image Comments': 'DCMCOM{IMAGE}',
 'Image Orientation (Patient)': '1.000000\\0.000000\\0.000000\\0.000000\\1.000000\\0.000000',
 'Image Position (Patient)': '-222.000000\\-106.000000\\1478.500000',
 'Image Type': 'DERIVED\\PRIMARY',
 'Imager Pixel Spacing': '0.781250\\0.781250',
 'Implementation Class UID': '1.2.826.0.1.3680043.2.1143.113349180606.201110071757

----

In [36]:
# import glob
# import os
# import SimpleITK as sitk

In [47]:
# mask_paths = sorted(glob.glob("/Users/anders1991/deepbreath/Data/Masks/*.nii"))
# scan_paths = sorted(glob.glob("/Users/anders1991/deepbreath/Data/Scans/*.nii"))

In [53]:
# sitk_image = sitk.ReadImage(scan_paths[0], sitk.sitkFloat32)
# image = sitk.GetArrayFromImage( sitk_image )

----

## Visualize

Unmasked:

In [11]:
from matplotlib import animation
from IPython.display import HTML

fig = plt.figure(figsize=(5,5))
i = np.linspace(1, scan.shape[0], num=scan.shape[0], dtype=int)-1
im = plt.imshow(scan[0,:,:], animated=True, cmap='jet',
               vmin=np.min(scan), vmax=np.max(scan))
plt.axis('off')

def animate(i):
    im.set_array(scan[i,:,:])
    return im

ani = animation.FuncAnimation(fig, animate, frames=scan.shape[0], 
                              interval=40, repeat=False)
plt.close()

In [12]:
HTML(ani.to_html5_video())

Masked:

In [13]:
scan_mask = np.copy(scan)
scan_mask[mask!=6] = np.nan

In [14]:
from matplotlib import animation
from IPython.display import HTML

fig = plt.figure(figsize=(5,5))
i = np.linspace(1, scan_mask.shape[0], num=scan_mask.shape[0], dtype=int)-1
im = plt.imshow(scan_mask[0,:,:], animated=True, cmap='jet',
                vmin=np.min(scan), vmax=np.max(scan))
plt.axis('off')

def animate(i):
    im.set_array(scan_mask[i,:,:])
    return im

ani = animation.FuncAnimation(fig, animate, frames=scan_mask.shape[0], 
                              interval=40, repeat=False)
plt.close()

In [15]:
HTML(ani.to_html5_video())

----